In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/animals5-100/OIP-LgbZBRDDbmMm4J5fubMBrAHaE7.jpeg
/kaggle/input/animals5-100/344.jpeg
/kaggle/input/animals5-100/OIP-OuLZ5AcJEYwHkKiKZipE7AHaFj.jpeg
/kaggle/input/animals5-100/991.jpeg
/kaggle/input/animals5-100/OIP-u2p7bbpwNEJlffZGqjCREAHaFj.jpeg
/kaggle/input/animals5-100/OIP-2h83tEtPUJS2PiCLCR6-aAHaFO.jpeg
/kaggle/input/animals5-100/OIP-oLc1YcLCOftRTwaS_HSKJAHaFj.jpeg
/kaggle/input/animals5-100/OIP-xRvRC9DO-CYTfYdH_fovAQHaFj.jpeg
/kaggle/input/animals5-100/OIP-41RZ-KLH76oHww2fOIfCfgHaF7.jpeg
/kaggle/input/animals5-100/OIP-ZJ8Fnnp_r8c1S1jeciGhbAHaHH.jpeg
/kaggle/input/animals5-100/1490.jpeg
/kaggle/input/animals5-100/OIP-Bxg1Dk55iBfftlQ2xYHpTQHaFk.jpeg
/kaggle/input/animals5-100/OIP-tKJhGTm2mobDFWJGYO-7EAHaE6.jpeg
/kaggle/input/animals5-100/1662.jpeg
/kaggle/input/animals5-100/OIP-a6-RGLrzpkRU774xRfXarwHaJ8.jpeg
/kaggle/input/animals5-100/1936.jpeg
/kaggle/input/animals5-100/166.jpeg
/kaggle/input/animals5-100/OIP-TCIVYjX_mY6ogmwDB8kk5AHaFj.jpeg
/kaggle/input/animals5-10

thư viện

In [2]:
import os  # Thư viện để làm việc với hệ thống tệp, dùng để truy cập thư mục dataset
import numpy as np  # Thư viện xử lý mảng số học, dùng cho tính toán và xử lý dữ liệu
import matplotlib.pyplot as plt  # Thư viện vẽ biểu đồ để trực quan hóa loss, accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Tạo generator để tiền xử lý và tăng cường dữ liệu ảnh
from tensorflow.keras.models import Sequential  # Mô hình tuần tự để xây MLP
from tensorflow.keras.layers import Dense, Flatten, Dropout  # Các tầng của mạng nơ-ron: Dense, Flatten (chuyển ảnh thành vector), Dropout (ngăn overfitting)
from tensorflow.keras.optimizers import Adam  # Optimizer Adam để tối ưu hóa mô hình
from tensorflow.keras.callbacks import EarlyStopping  # Callback để dừng huấn luyện sớm nếu không cải thiện
from sklearn.metrics import classification_report, confusion_matrix  # Công cụ đánh giá hiệu suất mô hình
import seaborn as sns  # Thư viện vẽ heatmap cho ma trận nhầm lẫn

2025-10-19 10:37:26.946805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760870247.368918      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760870247.472199      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tiền xử lý

In [3]:
# Định nghĩa các tham số cơ bản
img_height = 64  # Chiều cao ảnh sau khi resize (64 pixels để giảm kích thước input cho MLP)
img_width = 64  # Chiều rộng ảnh sau khi resize
batch_size = 32  # Kích thước batch để huấn luyện (số ảnh xử lý mỗi lần)
epochs = 20  # Số epoch tối đa (số lần lặp qua toàn bộ dữ liệu train)

# Đường dẫn đến dataset trên Kaggle
data_dir = '/kaggle/input/animals5-100/'  # Đường dẫn dựa trên tab Data, chứa các subfolder cho từng lớp

# Data augmentation và generator cho train/validation
train_datagen = ImageDataGenerator(  # Tạo đối tượng để tiền xử lý và tăng cường dữ liệu
    rescale=1./255,  # Chuẩn hóa pixel từ 0-255 về 0-1
    rotation_range=20,  # Xoay ngẫu nhiên ảnh trong khoảng 20 độ
    width_shift_range=0.2,  # Dịch chuyển ngang ngẫu nhiên tối đa 20% chiều rộng
    height_shift_range=0.2,  # Dịch chuyển dọc ngẫu nhiên tối đa 20% chiều cao
    shear_range=0.2,  # Biến dạng ảnh ngẫu nhiên
    zoom_range=0.2,  # Phóng to/thu nhỏ ngẫu nhiên
    horizontal_flip=True,  # Lật ngang ảnh ngẫu nhiên
    validation_split=0.2  # Chia 20% dữ liệu làm tập validation
)

# Tạo generator cho tập train
train_generator = train_datagen.flow_from_directory(  # Tạo generator cho tập train
    data_dir,  # Đường dẫn dataset
    target_size=(img_height, img_width),  # Resize ảnh về 64x64
    batch_size=batch_size,  # Số ảnh mỗi batch
    class_mode='categorical',  # Nhãn dạng one-hot (cho phân loại đa lớp)
    subset='training'  # Lấy tập train (80% dữ liệu)
)
num_classes = train_generator.num_classes  # Cập nhật số lớp dựa trên generator sau khi tạo
print("Số lớp thực tế:", num_classes)  # In số lớp để kiểm tra
print("Nhãn ánh xạ:", train_generator.class_indices)  # In ánh xạ giữa tên lớp và số thứ tự

# Tạo generator cho tập validation
validation_generator = train_datagen.flow_from_directory(  # Tạo generator cho tập validation
    data_dir,  # Đường dẫn dataset
    target_size=(img_height, img_width),  # Resize ảnh về 64x64
    batch_size=batch_size,  # Số ảnh mỗi batch
    class_mode='categorical',  # Nhãn dạng one-hot
    subset='validation'  # Lấy tập validation (20% dữ liệu)
)

Found 0 images belonging to 0 classes.
Số lớp thực tế: 0
Nhãn ánh xạ: {}
Found 0 images belonging to 0 classes.


Xây dựng mô hình MLP

In [4]:

model = Sequential([  # Tạo mô hình tuần tự
    Flatten(input_shape=(img_height, img_width, 3)),  # Chuyển ảnh 64x64x3 thành vector 1D (12288 phần tử)
    Dense(512, activation='relu'),  # Tầng đầu tiên với 512 nơ-ron, dùng ReLU, nhận vector từ Flatten
    Dropout(0.5),  # Loại bỏ ngẫu nhiên 50% nơ-ron để tránh overfitting
    Dense(256, activation='relu'),  # Tầng ẩn với 256 nơ-ron, dùng ReLU
    Dropout(0.5),  # Loại bỏ ngẫu nhiên 50% nơ-ron
    Dense(num_classes, activation='softmax')  # Tầng output với số nơ-ron bằng số lớp thực tế, dùng softmax
])

# Compile mô hình
model.compile(  # Cấu hình mô hình để huấn luyện
    optimizer=Adam(learning_rate=0.001),  # Dùng optimizer Adam với learning rate 0.001
    loss='categorical_crossentropy',  # Hàm mất mát cho phân loại đa lớp
    metrics=['accuracy']  # Theo dõi độ chính xác
)

model.summary()  # In cấu trúc mô hình (số tham số, tầng, v.v.)

# Callback để dừng sớm nếu không cải thiện
early_stopping = EarlyStopping(  # Tạo callback để dừng sớm
    monitor='val_loss',  # Theo dõi validation loss
    patience=5,  # Dừng sau 5 epoch nếu không cải thiện
    restore_best_weights=True  # Khôi phục trọng số tốt nhất
)  # Đóng dấu ngoặc cho EarlyStopping

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-10-19 10:37:48.095654: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 12288)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     6,291,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 0)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,423,296 (24.50 MB)

 Trainable params: 6,423,296 (24.50 MB)

 Non-trainable params: 0 (0.00 B)

huấn luyện

In [5]:
history = model.fit(  # Huấn luyện mô hình
    train_generator,  # Dữ liệu train từ generator
    steps_per_epoch=train_generator.samples // batch_size,  # Số bước mỗi epoch
    validation_data=validation_generator,  # Dữ liệu validation
    validation_steps=validation_generator.samples // batch_size,  # Số bước validation
    epochs=epochs,  # Số epoch tối đa
    callbacks=[early_stopping]  # Áp dụng early stopping
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0

đánh giá

In [ ]:
# Vẽ biểu đồ loss và accuracy
plt.figure(figsize=(12, 4))  # Tạo figure kích thước 12x4
plt.subplot(1, 2, 1)  # Biểu đồ 1: Loss
plt.plot(history.history['loss'], label='Train Loss')  # Vẽ loss của tập train
plt.plot(history.history['val_loss'], label='Val Loss')  # Vẽ loss của tập validation
plt.title('Loss')  # Tiêu đề
plt.legend()  # Hiển thị chú thích

plt.subplot(1, 2, 2)  # Biểu đồ 2: Accuracy
plt.plot(history.history['accuracy'], label='Train Acc')  # Vẽ accuracy của tập train
plt.plot(history.history['val_accuracy'], label='Val Acc')  # Vẽ accuracy của tập validation
plt.title('Accuracy')  # Tiêu đề
plt.legend()  # Hiển thị chú thích
plt.show()  # Hiển thị biểu đồ

# Đánh giá trên validation set
val_loss, val_acc = model.evaluate(validation_generator)  # Tính loss và accuracy trên tập validation
print(f"Validation Accuracy: {val_acc * 100:.2f}%")  # In độ chính xác (định dạng 2 chữ số thập phân)

# Dự đoán và báo cáo
Y_pred = model.predict(validation_generator)  # Dự đoán trên tập validation
y_pred = np.argmax(Y_pred, axis=1)  # Lấy lớp có xác suất cao nhất
y_true = validation_generator.classes  # Lấy nhãn thật từ generator

# In báo cáo phân loại (precision, recall, f1-score)
print(classification_report(y_true, y_pred, target_names=class_names))

# Vẽ ma trận nhầm lẫn
cm = confusion_matrix(y_true, y_pred)  # Tính ma trận nhầm lẫn
plt.figure(figsize=(8, 6))  # Tạo figure kích thước 8x6
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)  # Vẽ heatmap
plt.title('Confusion Matrix')  # Tiêu đề
plt.show()  # Hiển thị ma trận

# Lưu mô hình
model.save('/kaggle/working/mlp_animal_classifier.h5')  # Lưu mô hình vào file .h5
print("Mô hình đã được lưu tại /kaggle/working/mlp_animal_classifier.h5")  # Thông báo lưu thành công